# Check EDR Server Queries

Check that all queries supported by the EDR Server are functioning correctly. This is not really a test suite as (a) it's not automated and (b) it's neither complete nor checks error cases. What it does provide is some form of assurance that all expected queries are supported and running properly, and provides a reminder of how to call the different queries supported by the server.

## Setup

In [61]:
import json
import random

import requests
from shapely import wkt
from shapely.geometry import Point, Polygon

In [62]:
host = "http://localhost:8808"

## Checks

Now run through all the checks.

### Basic connectivity

Checks if the server is running. Should return a capabilities response (metadata about the server / host institute).

In [63]:
requests.get(host).content

b'{"title": "A dummy example", "description": "This is a dummy example of templating a capabilities request.", "links": [{"href": "http://www.example.org/edr/api", "hreflang": "en", "rel": "service", "type": "application/openapi+json;version=3.0", "title": ""}, {"href": "http://localhost:8808/conformance/", "hreflang": "en", "rel": "data", "type": "application/json", "title": ""}, {"href": "http://localhost:8808/collections/", "hreflang": "en", "rel": "data", "type": "application/json", "title": ""}], "keywords": ["Example", "Dummy"], "provider": {"name": "Galadriel", "url": ""}, "contact": {"email": "dummy@example.com", "phone": "07987 654321", "fax": "", "hours": "9 til 5", "instructions": "Don&#39;t", "address": "Over there", "postalCode": "ZZ99 9ZZ", "city": "Neverland", "stateorprovince": "", "country": "Wakanda"}}'

### Collections

Check the get collections request, which should return a JSON response of all the collections served.

In [64]:
query = f"{host}/collections"
response = requests.get(query).content
print(f"Number of collections: {len(json.loads(response)['collections'])}\n")
print(response)

Number of collections: 5

b'{"links": [{"href": "http://localhost:8808/collections", "rel": "self", "type": "application/json"}], "collections": [{"links": [{"href": "http://localhost:8808/collections/00001", "rel": "collection", "title": "Collection", "type": "application/json", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/cube", "rel": "data", "title": "Cube", "type": "cube", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/corridor", "rel": "data", "title": "Corridor", "type": "corridor", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/locations", "rel": "data", "title": "Locations", "type": "locations", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/items", "rel": "data", "title": "Items", "type": "items", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/area", "rel": "data", "title": "Area", "type": "area", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/

### Individual Collection

Check we can query an individual collection by retrieving the list of collection IDs from the previous query and requesting one of them at random. This should return a JSON response of a single collection.


In [65]:
collection_ids = [c["id"] for c in json.loads(response)['collections']]
random_collection_id = collection_ids[random.randint(0, len(collection_ids)-1)]
query = f"{host}/collections/{random_collection_id}"
response = requests.get(query).content

print(f"{collection_ids}\n")
print(response)

['00001', '00002', '00003', '00004', '00005']

b'{"links": [{"href": "http://localhost:8808/collections/00001", "rel": "collection", "title": "Collection", "type": "application/json", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/cube", "rel": "data", "title": "Cube", "type": "cube", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/corridor", "rel": "data", "title": "Corridor", "type": "corridor", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/locations", "rel": "data", "title": "Locations", "type": "locations", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/items", "rel": "data", "title": "Items", "type": "items", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/area", "rel": "data", "title": "Area", "type": "area", "hreflang": "en"}, {"href": "http://localhost:8808/collections/00001/position", "rel": "data", "title": "Position", "type": "position", "hreflang": "en"}, {"href": "h

### Locations

Check we can query an individual collection for its list of locations. This should return a JSON object of type `FeatureCollection` with one or more locations listed in it.

In [66]:
query = f"{host}/collections/00004/locations"
response = requests.get(query).content

print(f"Number of locations: {len(json.loads(response)['features'])}\n")
print(response)

Number of locations: 12

b'{"type": "FeatureCollection", "bbox": ["50.4", "-4.0", "55.5", "1.6"], "features": [{"type": "Feature", "id": "mast1", "geometry": {"type": "Point", "coordinates": ["-3.0", "55.5"]}, "properties": {"name": "Observing mast site 1", "datetime": "ScalarBounds(lower=datetime.datetime(2020, 8, 1, 12, 0, tzinfo=datetime.timezone.utc), upper=datetime.datetime(2020, 9, 1, 12, 0, tzinfo=datetime.timezone.utc))", "detail": "http://www.example.com/define/location/mast1", "description": "A point location over a timeseries", "extent": {"spatial": {"bbox": [-3.0, 55.5, -3.0, 55.5]}, "temporal": {"interval": [["ScalarBounds(lower=datetime.datetime(2020, 8, 1, 12, 0, tzinfo=datetime.timezone.utc), upper=datetime.datetime(2020, 9, 1, 12, 0, tzinfo=datetime.timezone.utc))"]]}}, "parameter_names": {"param7": {"type": "Parameter", "id": "param7", "description": {"en": "The seventh dummy parameter"}, "unit": {"label": {"en": "K"}, "symbol": {"value": "K", "type": "http://www.exam

### Individual Location

Check we can query an individual location in a collection. The response should be a JSON object of type `Domain` containing metadata for an individual location.

In [67]:
response = requests.get(f"{host}/collections/{random_collection_id}/locations").content
location_ids = [l["id"] for l in json.loads(response)['features']]
random_location_id = location_ids[random.randint(0, len(location_ids)-1)]
query = f"{host}/collections/{random_collection_id}/locations/{random_location_id}"
response = requests.get(query).content

print(f"{location_ids}\n")
print(response)

['50232']

b'{"domain": {"type": "Domain", "domainType": "Grid", "axes": {"x": {"start": -3.0, "stop": 0.0, "num": 100}, "y": {"start": 51.0, "stop": 54.0, "num": 100}, "t": {"values": ["2021-01-01T00:00:00+00:00", "2021-01-01T06:00:00+00:00", "2021-01-01T12:00:00+00:00", "2021-01-01T18:00:00+00:00"]}}, "referencing": [{"coordinates": ["x", "y"], "system": {"type": "GeographicCRS", "id": "http://www.example.com/define/crs/geog_crs"}}, {"coordinates": ["t"], "system": {"type": "TemporalRS", "calendar": "standard"}}]}, "parameters": {"param1": {"type": "Parameter", "id": "param1", "description": {"en": "The first dummy parameter"}, "unit": {"label": {"en": "m/s"}, "symbol": {"value": "m s-1", "type": "http://www.example.com/define/unit/ms-1"}}, "observedProperty": {"id": "http://www.example.com/phenom/dummy_1", "label": {"en": "Dummy 1"}}, "categoryEncoding": {"#000": 0.0, "#444": 20.0, "#888": 40.0, "#ccc": 60.0, "#fff": 80.0}, "preferredPalette": {"colors": ["#000", "#444", "#888", "#c

### Items

Check we can query an individual location for its list of items. This should return a JSON object of type `FeatureCollection` containing one or more individual items.

In [68]:
query = f"{host}/collections/{random_collection_id}/items"
response = requests.get(query).content

print(response)

b'{"type": "FeatureCollection", "numberReturned": 1, "numberMatched": 1, "timeStamp": "2022-06-16T14:21:59Z", "features": [{"type": "Feature", "id": "50232", "geometry": {"type": "Polygon", "coordinates": ["-3.0 51.0", "0.0 51.0", "0.0 54.0", "-3.0 54.0", "-3.0 51.0"]}, "properties": {"name": "Location 50232", "datetime": "ScalarBounds(lower=datetime.datetime(2021, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), upper=datetime.datetime(2021, 1, 1, 18, 0, tzinfo=datetime.timezone.utc))", "detail": "http://www.example.com/define/location/50232", "description": "A location"}, "links": [{"href": "http://localhost:8808/collections/00001/locations/50232", "rel": "data", "type": "application/cov+json", "hreflang": "en"}]}]}'


### Individual Item

Check we can make a data request to get data values that describe a single item, selected specifically from the dummy dataset for the sake of simplicity. The response should be data-containing (coverage)JSON of type `TiledNdArray`.

In [69]:
query = f"{host}/collections/00002/items/Parameter 4_0_0"
response = requests.get(query).content

print(response)

b'{"code": 404, "description": "Not Found", "message": "Item Parameter 4_0_0 was not found."}'


### Area

Check we can make a query to return data within a certain area. Note that this can mean either:
* "return the features that intersect with the area", or
* "return the data values for all features that are contained within the area"

The choice on how to implement this is left to the data interface. In the case of the dummy dataset, the first option is chosen. This should return a JSON response of type `Domain` containing one or more features that intersect with the polygon.

In [70]:
polygon = Polygon([(-0.9, 51.2), (0.3, 51.2), (0.3, 51.7), (-0.9, 51.7), (-0.9, 51.2)])
query_str = f"coords={wkt.dumps(polygon)}"
query = f"{host}/collections/00004/area?{query_str}"
response = requests.get(query).content

print(f'Feature IDs: {[f["id"] for f in json.loads(response)["features"]]}\n')
print(response)

Feature IDs: ['mast9', 'mast10', 'mast11']

b'{"type": "FeatureCollection", "bbox": ["50.4", "-4.0", "55.5", "1.6"], "features": [{"type": "Feature", "id": "mast9", "geometry": {"type": "Point", "coordinates": ["-0.8", "51.3"]}, "properties": {"name": "Observing mast site 9", "datetime": "ScalarBounds(lower=datetime.datetime(2020, 8, 1, 12, 0, tzinfo=datetime.timezone.utc), upper=datetime.datetime(2020, 9, 1, 12, 0, tzinfo=datetime.timezone.utc))", "detail": "http://www.example.com/define/location/mast9", "description": "A point location over a timeseries", "extent": {"spatial": {"bbox": [-0.8, 51.3, -0.8, 51.3]}, "temporal": {"interval": [["ScalarBounds(lower=datetime.datetime(2020, 8, 1, 12, 0, tzinfo=datetime.timezone.utc), upper=datetime.datetime(2020, 9, 1, 12, 0, tzinfo=datetime.timezone.utc))"]]}}, "parameter_names": {"param7": {"type": "Parameter", "id": "param7", "description": {"en": "The seventh dummy parameter"}, "unit": {"label": {"en": "K"}, "symbol": {"value": "K", "type

### Radius

The radius query is identical in intention and response to the area query, other than that the polygon is defined as a circle with a defined radius from a point.

**Not working!!** The radius is not being calculated correctly (for the case where units are like m, at least).

In [71]:
point = Point(-0.2, 51.5)
within = 0.5
units = "m"
query_str = f"coords={wkt.dumps(point)}&within={within}&within_units={units}"
query = f"{host}/collections/00005/radius?{query_str}"
response = requests.get(query).content

print(response)

b'{"code": 404, "description": "Not Found", "message": "No features located within provided Radius"}'


### Position

The position query is similar to radius and area, but the coordinates may only be an individual point location, or a list of points (in a `MultiPoint` geometry). This should return a JSON response of type `Domain`. 

In [72]:
point = Point(0.1, 51.5)
query_str = f"coords={wkt.dumps(point)}"
query = f"{host}/collections/00005/position?{query_str}"
response = requests.get(query).content

print(response)

b'{"domain": {"type": "Domain", "domainType": "Grid", "axes": {"x": {"values": [0.1]}, "y": {"values": [51.5]}, "z": {"values": [2, 10]}, "t": {"values": ["2020-08-01T12:00:00+00:00", "2020-08-02T12:00:00+00:00", "2020-08-03T12:00:00+00:00", "2020-08-04T12:00:00+00:00", "2020-08-05T12:00:00+00:00", "2020-08-06T12:00:00+00:00", "2020-08-07T12:00:00+00:00", "2020-08-08T12:00:00+00:00", "2020-08-09T12:00:00+00:00", "2020-08-10T12:00:00+00:00", "2020-08-11T12:00:00+00:00", "2020-08-12T12:00:00+00:00", "2020-08-13T12:00:00+00:00", "2020-08-14T12:00:00+00:00", "2020-08-15T12:00:00+00:00", "2020-08-16T12:00:00+00:00", "2020-08-17T12:00:00+00:00", "2020-08-18T12:00:00+00:00", "2020-08-19T12:00:00+00:00", "2020-08-20T12:00:00+00:00", "2020-08-21T12:00:00+00:00", "2020-08-22T12:00:00+00:00", "2020-08-23T12:00:00+00:00", "2020-08-24T12:00:00+00:00", "2020-08-25T12:00:00+00:00", "2020-08-26T12:00:00+00:00", "2020-08-27T12:00:00+00:00", "2020-08-28T12:00:00+00:00", "2020-08-29T12:00:00+00:00", "202